<a href="https://colab.research.google.com/github/HossamSaoud/MachineLearning_Notebooks/blob/main/EnsembleModels_RF_XGB_LGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
from keras.datasets import mnist
from sklearn.ensemble import AdaBoostClassifier,BaggingClassifier,StackingClassifier,RandomForestClassifier
from sklearn.metrics import classification_report,precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
import time
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import warnings
warnings.filterwarnings("ignore")

In [2]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()

11490434/11490434 [==============================] - 1s 0us/step


In [3]:
x_train_std=(x_train.reshape(x_train.shape[0],-1))/255.0
x_test_std=(x_test.reshape(x_test.shape[0],-1))/255.0

# Taking 10% of the data that includes all classes


In [4]:
classes=dict()
for i in range(len(np.unique(y_train))):
  classes[i]=np.where(y_train==i)[0]
print(classes)  
for key in classes:
  print(len(classes[key]))  

{0: array([    1,    21,    34, ..., 59952, 59972, 59987]), 1: array([    3,     6,     8, ..., 59979, 59984, 59994]), 2: array([    5,    16,    25, ..., 59983, 59985, 59991]), 3: array([    7,    10,    12, ..., 59978, 59980, 59996]), 4: array([    2,     9,    20, ..., 59943, 59951, 59975]), 5: array([    0,    11,    35, ..., 59968, 59993, 59997]), 6: array([   13,    18,    32, ..., 59982, 59986, 59998]), 7: array([   15,    29,    38, ..., 59963, 59977, 59988]), 8: array([   17,    31,    41, ..., 59989, 59995, 59999]), 9: array([    4,    19,    22, ..., 59973, 59990, 59992])}
5923
6742
5958
6131
5842
5421
5918
6265
5851
5949


In [5]:
x_train_sample=[]
y_train_sample=[]
for key in classes:
  length=round(len(classes[key])/10)
  for i,index in enumerate(classes[key]):
    if i>=length:
      break
    else:
      x_train_sample.append(x_train_std[index])
      y_train_sample.append(y_train[index])  

In [6]:
len(x_train_sample)

5999

# Applying ensemble techniques:

In [7]:
classifierbagging=BaggingClassifier(estimator=DecisionTreeClassifier(),n_estimators=10)
classifierbagging.fit(x_train_sample,y_train_sample)

BaggingClassifier(estimator=DecisionTreeClassifier())

In [8]:
y_predbagging=classifierbagging.predict(x_test_std)
scorebagging=round(precision_recall_fscore_support(y_test,y_predbagging,average='weighted')[2]*100,2)
print(f'Bagging_Fscore:{scorebagging}')

Bagging:88.54


In [9]:
estimators=[('SVC',SVC(kernel='sigmoid')),('DT',DecisionTreeClassifier(criterion='gini')),('LR',LogisticRegression())]

In [10]:
classifierstack=StackingClassifier(estimators=estimators,final_estimator=LogisticRegression())
classifierstack.fit(x_train_sample,y_train_sample)
y_predstack=classifierstack.predict(x_test_std)
scorestack=round(precision_recall_fscore_support(y_test,y_predstack,average='weighted')[2]*100,2)
accuracystack=round(accuracy_score(y_test,y_predstack)*100,2)
print(f'*Stack* accuracy= {accuracystack}%')
print(f'*stack* F1-score= {scorestack}%')

*Stack* accuracy= 90.65%
*stack* F1-score= 90.62%


In [11]:
classifierAda=AdaBoostClassifier(estimator=DecisionTreeClassifier(),n_estimators=100)
classifierAda.fit(x_train_sample,y_train_sample)
y_predAda=classifierAda.predict(x_test_std)
scoreAda=round(precision_recall_fscore_support(y_test,y_predAda,average='weighted')[2]*100,2)
accuracyAda=round(accuracy_score(y_test,y_predAda)*100,2)
print(f'*ADA* accuracy= {accuracyAda}%')
print(f'*ADA* F1-score= {scoreAda}%')

*ADA* accuracy= 78.41%
*ADA* F1-score= 78.42%


### Applying XGB,RF,LGBM on the data:



In [12]:
classifierLGBM=LGBMClassifier(n_estimators=100,random_state=0)
classifierLGBM.fit(x_train_sample,y_train_sample)
y_predLGBM=classifierLGBM.predict(x_test_std)

scoreLGBM=round(precision_recall_fscore_support(y_test,y_predLGBM,average='weighted')[2]*100,2)
accuracyLGBM=round(accuracy_score(y_test,y_predLGBM)*100,2)
print(f'*LGBM* accuracy= {accuracyLGBM}%')
print(f'*LGBM* F1-score= {scoreLGBM}%')

*LGBM* accuracy= 94.77%
*LGBM* F1-score= 94.77%


In [13]:
classifierRF=RandomForestClassifier(n_estimators=100,criterion='gini')
classifierRF.fit(x_train_sample,y_train_sample)
y_predRF=classifierRF.predict(x_test_std)
scoreRF=round(precision_recall_fscore_support(y_test,y_predRF,average='weighted')[2]*100,2)
accuracyRF=round(accuracy_score(y_test,y_predRF)*100,2)
print(f'*RF* accuracy= {accuracyRF}%')
print(f'*RF* F1-score= {scoreRF}%')

*RF* accuracy= 93.92%
*RF* F1-score= 93.91%


In [14]:
classifierXGB=XGBClassifier()
classifierXGB.fit(x_train_sample,y_train_sample)
y_predXGB=classifierXGB.predict(x_test_std)
scoreXGB=round(precision_recall_fscore_support(y_test,y_predXGB,average='weighted')[2]*100,2)
accuracyXGB=round(accuracy_score(y_test,y_predXGB)*100,2)
print(f'*XGB* accuracy= {accuracyXGB}%')
print(f'*XGB* F1-score= {scoreXGB}%')

*XGB* accuracy= 94.27%
*XGB* F1-score= 94.26%


# LightGBM gives us the best accuracy then we will use it on the whole dataset

In [15]:
classifierLGBM.fit(x_train_std,y_train)
y_predfinal=classifierLGBM.predict(x_test_std)
scoreXGB=round(precision_recall_fscore_support(y_test,y_predfinal,average='weighted')[2]*100,2)
accuracyXGB=round(accuracy_score(y_test,y_predfinal)*100,2)
print(f'*LGBM* accuracy= {accuracyXGB}%')
print(f'*LGBM* F1-score= {scoreXGB}%')

*LGBM* accuracy= 97.74%
*LGBM* F1-score= 97.74%
